 
# Federated Learning Training Plan: Execute Plan

Here we load and execute Plan and Model params created earlier in "Create Plan" notebook. 

This represents PySyft (python) worker.

In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
import torch as th
from torchvision import datasets, transforms
from syft.serde import protobuf
from syft_proto.execution.v1.plan_pb2 import Plan as PlanPB
from syft_proto.execution.v1.state_pb2 import State as StatePB
from syft import PlaceHolder
from syft.execution.state import State
import os
import numpy as np

sy.make_hook(globals())
# force protobuf serialization for tensors
hook.local_worker.framework = None

C:\Users\Vova\AppData\Local\conda\conda\envs\pysyft\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Vova\AppData\Local\conda\conda\envs\pysyft\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Vova\AppData\Local\conda\conda\envs\pysyft\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Vova\AppData\Loc

Setting up Sandbox...
Done!


Utility func that unserializes file contents into PySyft classes.
Note that we must know file contents beforehand to use specific protobuf class for deserialization.

In [2]:
def deserializeFromBin(worker, filename, pb):
    with open(filename, "rb") as f:
        bin = f.read()
    pb.ParseFromString(bin)
    return protobuf.serde._unbufferize(worker, pb)

def serializeToBinPb(worker, obj, filename):
    pb = protobuf.serde._bufferize(worker, obj)
    bin = pb.SerializeToString()
    print("Writing %s to %s/%s" % (obj.__class__.__name__, os.getcwd(), filename))
    with open(filename, "wb") as f:
        f.write(bin)

## Step 4: Unserialize Plan & Model

In [3]:
training_plan = deserializeFromBin(hook.local_worker, "tp_full.pb", PlanPB())
model_params_state = deserializeFromBin(hook.local_worker, "model_params.pb", StatePB())
# unwrap tensors from State
model_params = model_params_state.tensors()

print("Loaded plan (# of actions):", len(training_plan.role.actions))
print("Loaded torchscript plan code:", training_plan.torchscript.code)
print("Loaded params count:", len(model_params))

Loaded plan (# of actions): 39
Loaded torchscript plan code: def forward(self,
    argument_1: Tensor,
    argument_2: Tensor,
    argument_3: Tensor,
    argument_4: Tensor,
    argument_5: Tensor,
    argument_6: Tensor,
    argument_7: Tensor,
    argument_8: Tensor) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor, Tensor]:
  _0 = torch.matmul(argument_1, torch.t(argument_5))
  _1 = torch.add(_0, argument_6, alpha=1)
  _2 = torch.relu(_1)
  _3 = torch.add(torch.matmul(_2, torch.t(argument_7)), argument_8, alpha=1)
  _4 = torch.softmax(_3, 1, None)
  _5 = torch.mean(torch.mul(argument_2, torch.log(_4)), dtype=None)
  _6 = torch.neg(_5)
  _7 = torch.div(torch.sub(_4, argument_2, alpha=1), torch.mul(argument_3, CONSTANTS.c0))
  _8 = torch.matmul(_7, argument_7)
  _9 = torch.to(torch.gt(_1, 0), 6, False, False, None)
  _10 = torch.mul(_8, _9)
  _11 = torch.matmul(torch.t(_10), argument_1)
  _12 = torch.sum(_10, [0], False, dtype=None)
  _13 = torch.matmul(torch.t(_7), _2)
  _14 = torch.

## Step 5: Train!

Define the full training procedure that uses Plan as one training step on a batch of data. 

In [4]:
batch_size = 64
mnist = th.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True
)

def execute_training_plan(data, plan, model_params, epochs=1, batch_size=th.tensor(batch_size), lr=th.tensor(0.01)):
    for epoch in range(1, epochs+1):
        losses = []
        accuracies = []
        for batch_idx, (X, y) in enumerate(data):
            X = X.view(X.shape[0], -1)
            y_oh = th.nn.functional.one_hot(y, 10)
            *model_params, loss, acc = plan(X, y_oh, batch_size, lr, *model_params)
            losses.append(loss.item())
            accuracies.append(acc.item())
            if batch_idx % 100 == 0:
                print("Batch %d, loss: %f, accuracy: %f" % (batch_idx, loss, acc), end="\r")
        print('Epoch %d, avg loss: %f, avg training accuracy: %f' % (epoch, np.mean(losses), np.mean(accuracies)))
    return model_params

To show both variants of plans work, first we run training with "list of ops" Plan and get updated model weights,
then execute training with torchscript Plan starting with updated weights and get further updated weights :)

In [5]:
# Plain Plan
updated_model_params = execute_training_plan(mnist, training_plan, model_params)

Epoch 1, avg loss: 0.207543, avg training accuracy: 0.441714


In [6]:
# Torchscript Plan
# NOTE that execution point is `.torchscript` property
updated_model_params = execute_training_plan(mnist, training_plan.torchscript, updated_model_params)

Epoch 1, avg loss: 0.157949, avg training accuracy: 0.734275


## Step 6: Create Diff

Naive diff is just a difference between original model weights and updated model weights. 

In [7]:
diff = [ model_params[i] - updated_model_params[i] for i in range(len(model_params)) ]

print([ item.shape for item in diff ])

[torch.Size([392, 784]), torch.Size([392]), torch.Size([10, 392]), torch.Size([10])]


Let's wrap it in State to serialize.

In [8]:
diff_state = State(
    owner=hook.local_worker,
    state_placeholders=[PlaceHolder().instantiate(param) for param in diff]
)
serializeToBinPb(hook.local_worker, diff_state, 'diff.pb')


Writing State to e:\ml/diff.pb
